In [ ]:
!pip install imageio
!pip install git+https://github.com/tensorflow/docs
!pip install XlsxWriter

In [ ]:
import tensorflow as tf
import glob
import imageio
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow.keras as keras
from tensorflow.keras import layers
import time

from IPython import display
import tensorflow_docs.vis.embed as embed
import xlsxwriter

In [ ]:
BATCH_SIZE = 256

historical_images = tf.keras.preprocessing.image_dataset_from_directory(
    'drive/MyDrive/GanStick/dataset_eq/historical',
    batch_size=BATCH_SIZE,
    image_size=(56,56),
    shuffle=False,
    labels=None,
    smart_resize=True
)

future_images = tf.keras.preprocessing.image_dataset_from_directory(
    'drive/MyDrive/GanStick/dataset_eq/future',
    batch_size=BATCH_SIZE,
    image_size=(56,56),
    shuffle=False,
    labels=None,
    smart_resize=True
)

def process(image):
  image = tf.cast((image-127.5) / 127.5 ,tf.float32)
  return image

historical_dataset = historical_images.map(process)
future_dataset = future_images.map(process)

In [ ]:
def make_generator_model():
  gen_input_image = layers.Input(shape=(56, 56, 3))
  gen_input_noise = layers.Input(shape=(100,))

  image = layers.Conv2D(128, (5,5), strides=(2,2), padding='same', input_shape=[56, 56, 3])(gen_input_image)
  image = layers.BatchNormalization()(image)
  image = layers.LeakyReLU()(image)

  image = layers.Conv2D(64, (4,4), strides=(2,2), padding='same')(image)
  image = layers.BatchNormalization()(image)
  image = layers.LeakyReLU()(image)

  image = layers.Conv2D(32, (2,2), strides=(2,2), padding='same')(image)
  image = layers.BatchNormalization()(image)
  image = layers.LeakyReLU()(image)

  image = layers.Flatten()(image)
  image = layers.Dense(64)(image)
  image = layers.BatchNormalization()(image)
  image = layers.LeakyReLU()(image)

  image = layers.Dense(32)(image)
  image = layers.BatchNormalization()(image)
  image = layers.LeakyReLU()(image)
  image = layers.Dense(10, activation=tf.nn.tanh)(image)

  noise = layers.Concatenate(axis=1)([image, gen_input_noise])
  noise = layers.Dense(7*7*256, use_bias=False, input_shape=(110,))(noise)
  noise = layers.BatchNormalization()(noise)
  noise = layers.LeakyReLU()(noise)

  gen_image = layers.Reshape((7, 7, 256))(noise)
  
  gen_image = layers.Conv2DTranspose(128, (5, 5), strides=(2, 2), padding='same', use_bias=False)(gen_image)
  gen_image = layers.BatchNormalization()(gen_image)
  gen_image = layers.LeakyReLU()(gen_image)

  gen_image = layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False)(gen_image)
  gen_image = layers.BatchNormalization()(gen_image)
  gen_image = layers.LeakyReLU()(gen_image)

  gen_image = layers.Conv2DTranspose(32, (5, 5), strides=(2, 2), padding='same', use_bias=False)(gen_image)
  gen_image = layers.BatchNormalization()(gen_image)
  gen_image = layers.LeakyReLU()(gen_image)

  gen_image = layers.Conv2DTranspose(3, (5, 5), strides=(1, 1), padding='same', use_bias=False, activation='tanh')(gen_image)
  
  model = keras.Model(inputs=[gen_input_image, gen_input_noise], outputs=gen_image)

  return model

def make_discriminator_model():
  model = tf.keras.Sequential()
  model.add(layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same', input_shape=[56, 56, 3]))
  model.add(layers.LeakyReLU())
  model.add(layers.Dropout(0.3))

  model.add(layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
  model.add(layers.LeakyReLU())
  model.add(layers.Dropout(0.3))

  model.add(layers.Flatten())
  model.add(layers.Dense(1))

  return model

generator = make_generator_model()
generator.summary()
discriminator = make_discriminator_model()
discriminator.summary()

In [ ]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)
generator_optimizer = tf.keras.optimizers.Adam(0.0002)
discriminator_optimizer = tf.keras.optimizers.Adam(0.0002)

def generator_loss(fake_output):
  return cross_entropy(tf.ones_like(fake_output), fake_output)

def discriminator_loss(real_output, fake_output):
  real_loss = cross_entropy(tf.ones_like(real_output), real_output)
  fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
  total_loss = real_loss + fake_loss
  return total_loss

In [ ]:
checkpoint_dir = 'drive/MyDrive/GanStick/ConditionalDCGAN_check'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                  discriminator_optimizer=discriminator_optimizer,
                  generator=generator,
                  discriminator=discriminator)
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [ ]:
EPOCHS = 90
noise_dim = 100

@tf.function
def train_step(real_images, historical_images):
  current_batch_size = historical_images.shape[0]
  noise = tf.random.normal([current_batch_size, noise_dim])

  with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
    generated_images = generator([historical_images, noise], training=True)

    real_output = discriminator(real_images, training=True)
    fake_output = discriminator(generated_images, training=True)

    gen_loss = generator_loss(fake_output)
    disc_loss = discriminator_loss(real_output, fake_output)

  gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
  gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

  generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
  discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

  return gen_loss, disc_loss, real_images, generated_images, fake_output


def train(future_dataset, historical_dataset, epochs, last_epoch):
  for epoch in range(epochs):
    start = time.time()
    recent_epoch = epoch + last_epoch + 1
    data = []
    real_images = []
    generated_images = []
    batch_num = 0
    print('Start training for epoch {}'.format(recent_epoch))

    for future_images, historical_images in zip(future_dataset, historical_dataset):
      gen_loss, disc_loss, real_image, generated_image, fake_output = train_step(future_images, historical_images)
      data.append((gen_loss.numpy(), disc_loss.numpy(), fake_output.numpy()[0]))
      real_images.append(real_image.numpy()[0])
      generated_images.append(generated_image.numpy()[0])
      batch_num += 1
      if (batch_num % 10) == 0:
        print('Batch {} training finished'.format(batch_num))
    
    if (recent_epoch) % 10 == 0:
      checkpoint.save(file_prefix = checkpoint_prefix)

    save_result(data, real_images, generated_images, recent_epoch)
    display.clear_output(wait=True)
    print ('Time for epoch {} is {} sec'.format(recent_epoch, time.time()-start))
    print ('generator loss:', gen_loss.numpy())
    print ('disciminator loss:', disc_loss.numpy())


def save_result(data, real_images, generated_images, epoch_num):
  wb = xlsxwriter.Workbook(f'drive/MyDrive/GanStick/ConditionalDCGAN_result/epoch{epoch_num:03}.xlsx')
  os.makedirs(f'drive/MyDrive/GanStick/ConditionalDCGAN_result/epoch{epoch_num:03}/real', exist_ok=True)
  os.makedirs(f'drive/MyDrive/GanStick/ConditionalDCGAN_result/epoch{epoch_num:03}/generated', exist_ok=True)
  ws = wb.add_worksheet()
  ws.write_row(0, 0, ('Batch Index', 'Generator Loss', 'Discriminator Loss', 'Generated Image Prediction'))
  batch_num = 1
  for result, real_img, gene_img in zip(data, real_images, generated_images):
    ws.write_row(batch_num, 0, (batch_num, result[0], result[1], result[2]))
    save_img = (real_img * 127.5 + 127.5)
    save_img = PIL.Image.fromarray(np.uint8(save_img))
    save_img.save(f'drive/MyDrive/GanStick/ConditionalDCGAN_result/epoch{epoch_num:03}/real/{batch_num:03}.png') 
    save_img = (gene_img * 127.5 + 127.5)
    save_img = PIL.Image.fromarray(np.uint8(save_img))
    save_img.save(f'drive/MyDrive/GanStick/ConditionalDCGAN_result/epoch{epoch_num:03}/generated/{batch_num:03}.png')
    batch_num += 1
  wb.close()

In [ ]:
train(future_dataset, historical_dataset, 90, 0)